# 给定一个净值序列，计算年化收益、最大回撤、夏普比率

##  数据准备：净值序列

~~~ 
以HS300的收盘价，作为策略的净值

~~~

获取数据准备工作

1. 安装JQDdata库，[JQData使用说明](https://www.joinquant.com/help/api/help?name=JQData#%E5%A6%82%E4%BD%95%E5%AE%89%E8%A3%85%E4%BD%BF%E7%94%A8JQData)。
    
2. 在聚宽申请账号，并且[申请开通JQData权限](https://www.joinquant.com/default/index/sdk?utm_campaign=JQData%E7%94%B3%E8%AF%B7&utm_medium=%E7%BD%91%E9%A1%B5&utm_source=%E8%81%9A%E5%AE%BD&gio_link_id=xRxqAjP5)。

3. Python版本: Python3.7

In [1]:
# 从聚宽JQData获取HS300从2019/1/1至2019年底，每个交易日的收盘价
# 导入JQData库
from jqdatasdk import *
import pandas as pd


In [12]:
# ID是申请时所填写的手机号；Password为聚宽官网登录密码
# auth('ID','Password') 
auth('13811550483','security421S')
pd.set_option('display.max_rows', 10)

In [3]:
# 获取HS300每日收盘价，时间：2015-01-01至2018-12-31
df = get_price('000300.XSHG', start_date='2019-01-01', end_date='2019-12-31', frequency='daily', fields=['close'], skip_paused=False, fq='pre')

# 净值统计：以第一个交易日收盘价为参考，计算每日净值
first_close = df['close'][0]
net_values = df['close']/ first_close
net_values

2019-01-02    1.000000
2019-01-03    0.998417
2019-01-04    1.022337
2019-01-07    1.028543
2019-01-08    1.026321
                ...   
2019-12-25    1.343935
2019-12-26    1.355762
2019-12-27    1.354429
2019-12-30    1.374499
2019-12-31    1.379534
Name: close, Length: 244, dtype: float64

## 根据日净值分别计算年化收益、最大回撤、夏普比率

###  计算年化收益

- 计算公式

    $ Total Annualized Returns=Rp= ((1+P)^{\frac{245}{n}−1}) * 100\%  $

    $ P=策略收益 $

    $ n=策略执行天数 $

In [4]:
def compute_annual_profit(days, net_value):
    """
    计算年化收益
    """

    annual_profit = 0
    # 交易日数大于0，才计算年化收益
    if days > 0:
        # 计算年数
        years = days / 245
        # 计算年化收益
        annual_profit = pow(net_value, 1 / years) - 1

    # 将年化收益转化为百分数，保留两位小数
    annual_profit = round(annual_profit * 100, 2)

    return annual_profit

In [5]:
print('年化收益率:', compute_annual_profit(len(df), net_values[-1]), "%")

年化收益率: 38.14 %


### 计算最大回撤

~~~
描述策略可能出现的最糟糕的情况，最极端可能的亏损情况。

~~~


- 计算公式

    $ Max Drawdown=Max(Px−Py)/Px $

    $ Px,Py=策略某日净值，y>x $ 

In [6]:
def compute_drawdown(net_values):
    """
    计算最大回撤
    :param net_values: 净值列表
    """
    # 最大回撤初始值设为0
    max_drawdown = 0
    index = 0
    # 双层循环找出最大回撤
    for net_value in net_values:
        # 计算从当前开始直到结束，和当前净值相比的最大回撤
        for sub_net_value in net_values[index:]:
            # 计算回撤
            drawdown = 1 - sub_net_value / net_value
            # 如果当前的回撤大于已经计算的最大回撤，则当前回撤作为最大回撤
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        index += 1
    return max_drawdown


In [7]:
max_drawdown = compute_drawdown(list(net_values))
print('最大回撤:', round(max_drawdown * 100, 2), "%")

最大回撤: 13.49 %


### 计算夏普比率

~~~
表示每承受一单位总风险，会产生多少的超额报酬，可以同时对策略的收益与风险进行综合考虑。
~~~

- 计算公式

    $ Sharpe Ratio = \frac{R_p - R_f}{\sigma_p \times \sqrt 245  } $ 
    
    $ R_p = 策略年化收益率  $
    
    $ R_f = 无风险利率  $
    
    $ \sigma_p = 策略收益波动率  $
    
    


In [9]:
def compute_sharpe_ratio(net_values):
    """
    计算夏普比率
    :param net_values: 净值列表
    """

    # 总交易日数
    trading_days = len(net_values)
    # 所有收益的DataFrame
    profit_df = pd.DataFrame(columns={'profit'})
    # 收益之后，初始化为第一天的收益
    profit_df.loc[0] = {'profit': round((net_values[0] - 1) * 100, 2)}
    # 计算每天的收益
    for index in range(1, trading_days):
        # 计算每日的收益变化
        profit = (net_values[index] - net_values[index - 1]) / net_values[index - 1]
        profit = round(profit * 100, 2)
        profit_df.loc[index] = {'profit': profit}

    

    # 计算当日收益标准差
    profit_std = pow(profit_df['profit'].var(), 1 / 2)

    # 年化收益
    annual_profit = compute_annual_profit(trading_days, net_values[-1])

    # 夏普比率
    sharpe_ratio = (annual_profit - 4.75) / (profit_std * pow(245, 1 / 2))

    return sharpe_ratio



In [10]:
sharp = compute_sharpe_ratio(list(net_values))
print('夏普比率:', round(sharp,3))

夏普比率: 1.711


## 习题：根据HS300周净值序列， 分别计算年化收益、最大回撤、夏普比率

In [13]:
# 样例： 从聚宽JQData获取HS300的周K数据

# ID是申请时所填写的手机号；Password为聚宽官网登录密码
# auth('ID','Password') 

pd.set_option('display.max_rows', 10)

# 获取HS300周K收盘价
df = get_price('000300.XSHG', start_date='2019-01-01', end_date='2019-12-31', frequency='5d', fields=['close'], skip_paused=False, fq='pre')
first_close = df['close'][0]
net_values = df['close']/ first_close
net_values

2019-01-08    1.000000
2019-01-15    1.026344
2019-01-22    1.031374
2019-01-29    1.047994
2019-02-12    1.092739
                ...   
2019-12-04    1.263189
2019-12-11    1.280556
2019-12-18    1.323221
2019-12-25    1.309469
2019-12-31    1.344155
Name: close, Length: 49, dtype: float64